In [2]:
import numpy as np
import matplotlib.pyplot as plt
from emnist import extract_training_samples
from sklearn.utils import shuffle
from functools import partial
import time

from vertices_generator import vertices
from kernel import Gaussian_kernel
from mdwsvm import mdwsvm
from mdwsvm_ad import mdwsvm_ad
from one_class_svm import one_class_svm
from hybrid import hybrid
from metric import within_class_error

# Loading Training, Validation, and Test Data

In [3]:
# Load data
digits_images, digits_labels = extract_training_samples('digits')
letters_images, letters_labels = extract_training_samples('byclass')

# Get number 
mask_1 = (digits_labels == 1)
digits_images_1 = digits_images[mask_1]
digits_labels_1 = digits_labels[mask_1]

mask_3 = (digits_labels == 3)
digits_images_3 = digits_images[mask_3]
digits_labels_3 = digits_labels[mask_3]

mask_5 = (digits_labels == 5)
digits_images_5 = digits_images[mask_5]
digits_labels_5 = digits_labels[mask_5]

mask_7 = (digits_labels == 7)
digits_images_7 = digits_images[mask_7]
digits_labels_7 = digits_labels[mask_7]


# Get letter u, v, w, x, y, z
mask_uvwxyz = (letters_labels == 56) | (letters_labels == 57) | (letters_labels == 58) | (letters_labels == 59) | (letters_labels == 60) | (letters_labels == 61)
letters_images = letters_images[mask_uvwxyz]
letters_labels = letters_labels[mask_uvwxyz]
print(len(letters_labels))

16349


In [4]:
# Get training and testing data
X_train = np.zeros((800,28,28))
y_train = np.zeros((800), dtype=int)
X_val = np.zeros((8000,28,28))
y_val = np.zeros((8000), dtype=int)
X_test = np.zeros((8000,28,28))
y_test = np.zeros((8000), dtype=int)

# 800 digits normalized training data 
X_train[0:150,:,:] = digits_images_1[0:150,:,:] / 255
X_train[150:300,:,:] = digits_images_3[0:150,:,:] / 255
X_train[300:550,:,:] = digits_images_5[0:250,:,:] / 255
X_train[550:800,:,:] = digits_images_7[0:250,:,:] / 255
X_train = X_train.reshape(800,784).T 
# 800 digits training label
y_train[0:150] = digits_labels_1[0:150] - 1
y_train[150:300] = digits_labels_3[0:150] - 2
y_train[300:550] = digits_labels_5[0:250] - 3
y_train[550:800] = digits_labels_7[0:250] - 4

# Used for hybrid
# Get 400 digits for validation X
X_val[0:100,:,:] = digits_images_1[1000:1100,:,:] / 255
X_val[100:200,:,:] = digits_images_3[1000:1100,:,:] / 255
X_val[200:300,:,:] = digits_images_5[1000:1100,:,:] / 255
X_val[300:400,:,:] = digits_images_7[1000:1100,:,:] / 255
# 400 digits validation label
y_val[0:100] = digits_labels_1[1000:1100] - 1
y_val[100:200] = digits_labels_3[1000:1100] - 2
y_val[200:300] = digits_labels_5[1000:1100] - 3
y_val[300:400] = digits_labels_7[1000:1100] - 4
# Get 7600 lowercase letters
X_val[400:8000,:,:] = letters_images[0:7600,:,:] / 255
y_val[400:8000] = letters_labels[0:7600]
# Get true y label to calculate hybrid error
y_val_true_hybrid = -np.ones((8000), dtype=int)
y_val_true_hybrid[0:400] = y_val[0:400]
# Get true y label to calculate mdwsvm_ad error
y_val_true_mdwsvm_ad = 4 * np.ones((8000), dtype=int)
y_val_true_mdwsvm_ad[0:400] = y_val[0:400]
# 400 digits and 7600 letters normalized data
X_val = X_val.reshape(8000,784).T

# Get 400 digits for test X
X_test[0:100,:,:] = digits_images_1[1100:1200,:,:] / 255
X_test[100:200,:,:] = digits_images_3[1100:1200,:,:] / 255
X_test[200:300,:,:] = digits_images_5[1100:1200,:,:] / 255
X_test[300:400,:,:] = digits_images_7[1100:1200,:,:] / 255
# 400 digits test label
y_test[0:100] = digits_labels_1[1100:1200] - 1
y_test[100:200] = digits_labels_3[1100:1200] - 2
y_test[200:300] = digits_labels_5[1100:1200] - 3
y_test[300:400] = digits_labels_7[1100:1200] - 4
# Get 7600 lowercase letters
X_test[400:8000,:,:] = letters_images[8000:15600,:,:] / 255
y_test[400:8000] = letters_labels[8000:15600]
# Get true y label to calculate hybrid error
y_test_true_hybrid = -np.ones((8000), dtype=int)
y_test_true_hybrid[0:400] = y_test[0:400]
# Get true y label to calculate mdwsvm_ad error
y_test_true_mdwsvm_ad = 4 * np.ones((8000), dtype=int)
y_test_true_mdwsvm_ad[0:400] = y_test[0:400]
# 400 digits and 7600 letters normalized data
X_test = X_test.reshape(8000,784).T

# y_test: 0,1,2,3,56-61
# y_test_true_hybrid: -1,0,1,2,3
# y_test_true_mdwsvm_ad: 0,1,2,3,4

In [22]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2, 3]), array([150, 150, 250, 250], dtype=int64))

In [23]:
np.unique(y_val, return_counts=True)

(array([ 0,  1,  2,  3, 56, 57, 58, 59, 60, 61]),
 array([ 100,  100,  100,  100, 1317, 1368, 1271, 1312, 1085, 1247],
       dtype=int64))

In [24]:
np.unique(y_test, return_counts=True)

(array([ 0,  1,  2,  3, 56, 57, 58, 59, 60, 61]),
 array([ 100,  100,  100,  100, 1339, 1327, 1255, 1315, 1084, 1280],
       dtype=int64))

In [25]:
X_train.shape

(784, 800)

Error metric: use metric.py

# MDWSVM

## Cross Validation for MDWSVM

Only use the training data to do k-fold cross validation, and choose the best hyperparameter C based on average cross validation error (sum of all cross validation errors for one C divided by k). Use alpha = 0.5.

In [26]:
c_values = [2**i for i in range(-3,13)]
w1 = vertices(4)
size = X_train.shape[1]
num_folds = 5

X_train_new, y_train_new = shuffle(X_train.T, y_train, random_state=42)
X_train_new = X_train_new.T
folder_size = int(size / num_folds)
# Loop over each value of c and perform cross-validation
best_c = 0
best_score = -1
for c in c_values:
    scores = np.zeros(num_folds)
    # Perform cross-validation and calculate the average score
    for i in range(num_folds):
        # Get testing set    
        testx = X_train_new[:, i*folder_size:(i+1)*folder_size]
        testy = y_train_new[i*folder_size:(i+1)*folder_size]
        # Get training set    
        trainx = np.hstack((X_train_new[:, 0:(i)*folder_size], X_train_new[:, (i+1)*folder_size:size]))
        trainy = np.hstack((y_train_new[0:(i)*folder_size], y_train_new[(i+1)*folder_size:size]))
        method = mdwsvm(trainx, trainy, w1, c)
        
        pred_y = method.predict(testx)
        score = 1 - within_class_error(testy, pred_y)
        scores[i] = score
    # Check if the current value of c is the best so far
    avg_score = np.mean(scores)
    print(avg_score)
    if avg_score > best_score:
        best_c = c
        best_score = avg_score

0.7219278257500831
0.8804499317653143
0.9063206637381365
0.9238592444711161
0.9106620778738762
0.9003943726964261
0.9136824123822656
0.9125231552450671
0.8859543690333105
0.8792666353572519
0.8698996777188814
0.8820229014921457
0.8646591198657886
0.8505137407449042
0.8477638929055858
0.8526854615330368


In [27]:
print('The best c is', best_c)
print('The best score is', best_score)

The best c is 1
The best score is 0.07614075552888389


## Test Error for MDWSVM

1. Use the best C to train the model again using training data
2. Evaluate the model on the test data.

In [5]:
best_c = 1
model1 = mdwsvm(X_train, y_train, w1, best_c)
y_pred_1 = model1.predict(X_test)

NameError: name 'w1' is not defined

In [ ]:
print('The error is', within_class_error(y_test, y_pred_1))

The error is 0.636


# Hybrid

## Grid Search for Hybrid Method

Find hyperparameters for hybrid: v and sigma2. 
1. Use C find in MDWSVM. 
2. For each pair of v and sigma2, train the model use training data and evaluate it on the validation data. Return the validation error. 
3. Choose the best pair of hyperparameters that has lowest validation error.
4. Record the running time of the whole training process for each pair of v and sigma2. (Use later to compare the speed of this method and our method)

Range:
1. v in [0.1, 0.9] step = 0.1
2. sigma2 in [1,5,10,15,20,25,30,40,50]

In [4]:
v_values = [0.1, 0.3, 0.5, 0.7, 0.9]
sigma2_values = [0.001, 0.01, 0.05, 0.1, 0.3, 0.6, 1, 1.5]
w1 = vertices(4)

best_c = 1
best_v = 0
best_sigma2 = 0
best_score = -1
time_record_1 = []
for v in v_values:
    for s in sigma2_values:
        print('-'*5+'Start: v=%.2f'%v+' sigma2=%.3f'%s+'-'*5)
        k = partial(Gaussian_kernel, sigma2=s)
        st = time.time()
        y_pred = hybrid(X_train, y_train, X_val, v, w1, best_c, k)
        et = time.time()
        time_record_1.append(et - st)
        score = 1 - within_class_error(y_val_true_hybrid, y_pred)
        print(score)
        if score > best_score:
            best_v = v
            best_sigma2 = s
            best_score = score


-----Start: v=0.10 sigma2=0.00-----
0.40549999999999997
-----Start: v=0.10 sigma2=0.01-----
0.644157894736842
-----Start: v=0.10 sigma2=0.05-----
0.7029473684210527
-----Start: v=0.10 sigma2=0.10-----
0.34055263157894733
-----Start: v=0.10 sigma2=0.30-----
0.630578947368421
-----Start: v=0.10 sigma2=0.60-----
0.6400526315789474
-----Start: v=0.10 sigma2=1.00-----
0.664
-----Start: v=0.10 sigma2=1.50-----
0.6759999999999999
-----Start: v=0.30 sigma2=0.00-----
0.5382631578947368
-----Start: v=0.30 sigma2=0.01-----
0.7340263157894736
-----Start: v=0.30 sigma2=0.05-----
0.7211578947368421
-----Start: v=0.30 sigma2=0.10-----
0.5819736842105263
-----Start: v=0.30 sigma2=0.30-----
0.632578947368421
-----Start: v=0.30 sigma2=0.60-----
0.6400526315789474
-----Start: v=0.30 sigma2=1.00-----
0.664
-----Start: v=0.30 sigma2=1.50-----
0.6759999999999999
-----Start: v=0.50 sigma2=0.00-----
0.6271315789473684
-----Start: v=0.50 sigma2=0.01-----
0.6486315789473684
-----Start: v=0.50 sigma2=0.05-----
0

In [5]:
print('The best v is', best_v)
print('The best sigma2 is', best_sigma2)
print('The best score is', best_score)

The best v is 0.3
The best sigma2 is 0.01
The best score is 0.7340263157894736


## Test error for Hybrid Model

1. Use the best v, sigma2 and C to train the model using training data
2. Evaluate the model on the test data

In [6]:
best_sigma2 = 0.01
best_v = 0.3
best_c = 1
w1 = vertices(4)
best_k = partial(Gaussian_kernel, sigma2=best_sigma2)
y_pred_2 = hybrid(X_train, y_train, X_test, best_v, w1, best_c, best_k)

In [7]:
print('The error is', within_class_error(y_test_true_hybrid, y_pred_2))

The error is 0.3654736842105263


# MDWSVM_AD

## Grid Search for MDWSVM_AD Method

Find best hyperparameters
1. For each pair of v, sigma2 and C, train the model use training data and evaluate it on the validation data. Return the validation error. 
2. Choose the best pair of hyperparameters that has lowest validation error.
3. Record the running time of the whole training process for each pair of v, sigma2 and C. (This method should be faster than hybrid and should have a little bit higher accuracy when using the best hyperparameters`)

Range:
1. v in [0.6~0.95] step=0.05
2. sigma in [1, 5] + [10~20] with step=0.5
3. C in [1,5,8,9,10,11]

In [5]:
v_values = [0.1, 0.3, 0.5, 0.7, 0.9]
sigma2_values = [10, 12, 14, 16, 18, 55]
C_values = [1,5,8,9,10,11]
w2 = vertices(5)

best_v_2 = 0
best_sigma2_2 = 0
best_c_2 = 0
best_score_2 = -1
time_record_2 = []
for v in v_values:
    for s in sigma2_values:
        for c in C_values:
            print('-'*5+'Start: v=%.3f'%v+' sigma2=%.3f'%s+' c=%.3f'%c+'-'*5)
            k = partial(Gaussian_kernel, sigma2=s)
            st = time.time()
            method = mdwsvm_ad(X_train, y_train, w2, c, v, k)
            y_pred = method.predict(X_val)
            et = time.time()
            time_record_2.append(et - st)
            score = 1 - within_class_error(y_val_true_mdwsvm_ad, y_pred)
            print(score)
            if score > best_score_2:
                best_v_2 = v
                best_sigma2_2 = s
                best_c_2 = c
                best_score_2 = score


-----Start: v=0.100 sigma2=10.000 c=1.000-----
0.7824473684210527
-----Start: v=0.100 sigma2=10.000 c=5.000-----
0.8016578947368421
-----Start: v=0.100 sigma2=10.000 c=8.000-----
0.7824473684210527
-----Start: v=0.100 sigma2=10.000 c=9.000-----
0.8017105263157895
-----Start: v=0.100 sigma2=10.000 c=10.000-----
0.7824736842105263
-----Start: v=0.100 sigma2=10.000 c=11.000-----
0.7824736842105263
-----Start: v=0.100 sigma2=12.000 c=1.000-----
0.8743947368421052
-----Start: v=0.100 sigma2=12.000 c=5.000-----
0.8437368421052631
-----Start: v=0.100 sigma2=12.000 c=8.000-----
0.8398684210526316
-----Start: v=0.100 sigma2=12.000 c=9.000-----
0.8698947368421053
-----Start: v=0.100 sigma2=12.000 c=10.000-----
0.8458157894736842
-----Start: v=0.100 sigma2=12.000 c=11.000-----
0.8698947368421053
-----Start: v=0.100 sigma2=14.000 c=1.000-----
0.8301315789473684
-----Start: v=0.100 sigma2=14.000 c=5.000-----
0.8282631578947368
-----Start: v=0.100 sigma2=14.000 c=8.000-----
0.8263421052631579
-----S

In [ ]:
print('The best v is', best_v_2)
print('The best sigma2 is', best_sigma2_2)
print('The best C is', best_c_2)
print('The best score is', best_score_2)

## Test Error for MDWSVM_AD Method

Use the best v, sigma2, and C to train the model on the training data and evaluate it on the test data.

In [8]:
# v=0.100 sigma2=12.000 c=1.000 score:0.8743947368421052
best_v_2 = 0.1
best_sigma2_2 = 12
best_c_2 = 1
w2 = vertices(5)
best_k_2 = partial(Gaussian_kernel, sigma2=best_sigma2_2)
model3 = mdwsvm_ad(X_train, y_train, w2, best_c_2, best_v_2, best_k_2)
y_pred_3 = model3.predict(X_test)

In [9]:
print('The error is', within_class_error(y_test_true_mdwsvm_ad, y_pred_3))

The error is 0.13728947368421052


# For each method, the evaluation on the test data should return two tables.

1. one is about for each digit and letter, the percentage that it is assigned to each class label (5-8 and letter class) (like the table in CS++paper)
2. one is a table for all methods: two rows: one row is what percentage of all the digits are correctly classified, one row is what percentage of all the letters are correctly classified.

In [10]:
# number: 1-0,3-1,5-2,7-3
# letter: u-56, v-57, w-58, x-59, y-60, z-61
# hybrid:-1 0 1 2 3
# ad: 0 1 2 3 4
total_u = len(y_test[y_test == 56])
total_v = len(y_test[y_test == 57])
total_w = len(y_test[y_test == 58])
total_x = len(y_test[y_test == 59])
total_y = len(y_test[y_test == 60])
total_z = len(y_test[y_test == 61])
total_num = 100

# Create plot for MDWSVM:
result_matrix = np.zeros((10,4)) # 10 input class, 4 predict class
for i in range(8000):
    if(y_test[i] == 56): # If its true label is u
        if(y_pred_1[i] == 0):
            result_matrix[0,0] += 1
        if(y_pred_1[i] == 1):
            result_matrix[0,1] += 1
        if(y_pred_1[i] == 2):
            result_matrix[0,2] += 1
        if(y_pred_1[i] == 3):
            result_matrix[0,3] += 1
    if(y_test[i] == 57): # If its true label is v
        if(y_pred_1[i] == 0):
            result_matrix[1,0] += 1
        if(y_pred_1[i] == 1):
            result_matrix[1,1] += 1
        if(y_pred_1[i] == 2):
            result_matrix[1,2] += 1
        if(y_pred_1[i] == 3):
            result_matrix[1,3] += 1
    if(y_test[i] == 58): # If its true label is w
        if(y_pred_1[i] == 0):
            result_matrix[2,0] += 1
        if(y_pred_1[i] == 1):
            result_matrix[2,1] += 1
        if(y_pred_1[i] == 2):
            result_matrix[2,2] += 1
        if(y_pred_1[i] == 3):
            result_matrix[2,3] += 1
    if(y_test[i] == 59): # If its true label is x
        if(y_pred_1[i] == 0):
            result_matrix[3,0] += 1
        if(y_pred_1[i] == 1):
            result_matrix[3,1] += 1
        if(y_pred_1[i] == 2):
            result_matrix[3,2] += 1
        if(y_pred_1[i] == 3):
            result_matrix[3,3] += 1
    if(y_test[i] == 60): # If its true label is y
        if(y_pred_1[i] == 0):
            result_matrix[4,0] += 1
        if(y_pred_1[i] == 1):
            result_matrix[4,1] += 1
        if(y_pred_1[i] == 2):
            result_matrix[4,2] += 1
        if(y_pred_1[i] == 3):
            result_matrix[4,3] += 1
    if(y_test[i] == 61): # If its true label is z
        if(y_pred_1[i] == 0):
            result_matrix[5,0] += 1
        if(y_pred_1[i] == 1):
            result_matrix[5,1] += 1
        if(y_pred_1[i] == 2):
            result_matrix[5,2] += 1
        if(y_pred_1[i] == 3):
            result_matrix[5,3] += 1
    if(y_test[i] == 0): # If its true label is 1
        if(y_pred_1[i] == 0):
            result_matrix[6,0] += 1
        if(y_pred_1[i] == 1):
            result_matrix[6,1] += 1
        if(y_pred_1[i] == 2):
            result_matrix[6,2] += 1
        if(y_pred_1[i] == 3):
            result_matrix[6,3] += 1
    if(y_test[i] == 1): # If its true label is 3
        if(y_pred_1[i] == 0):
            result_matrix[7,0] += 1
        if(y_pred_1[i] == 1):
            result_matrix[7,1] += 1
        if(y_pred_1[i] == 2):
            result_matrix[7,2] += 1
        if(y_pred_1[i] == 3):
            result_matrix[7,3] += 1
    if(y_test[i] == 2): # If its true label is 5
        if(y_pred_1[i] == 0):
            result_matrix[8,0] += 1
        if(y_pred_1[i] == 1):
            result_matrix[8,1] += 1
        if(y_pred_1[i] == 2):
            result_matrix[8,2] += 1
        if(y_pred_1[i] == 3):
            result_matrix[8,3] += 1
    if(y_test[i] == 3): # If its true label is 7
        if(y_pred_1[i] == 0):
            result_matrix[9,0] += 1
        if(y_pred_1[i] == 1):
            result_matrix[9,1] += 1
        if(y_pred_1[i] == 2):
            result_matrix[9,2] += 1
        if(y_pred_1[i] == 3):
            result_matrix[9,3] += 1

print(result_matrix)
print(np.sum(result_matrix))

NameError: name 'y_pred_1' is not defined

In [ ]:
# Create plot for hybrid:
result_matrix2 = np.zeros((10,5)) # 10 input class, 5 predict class
for i in range(8000):
    if(y_test[i] == 56): # If its true label is u
        if(y_pred_2[i] == -1):
            result_matrix2[0,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix2[0,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix2[0,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix2[0,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix2[0,3] += 1
    if(y_test[i] == 57): # If its true label is v
        if(y_pred_2[i] == -1):
            result_matrix2[1,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix2[1,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix2[1,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix2[1,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix2[1,3] += 1
    if(y_test[i] == 58): # If its true label is w
        if(y_pred_2[i] == -1):
            result_matrix2[2,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix2[2,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix2[2,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix2[2,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix2[2,3] += 1
    if(y_test[i] == 59): # If its true label is x
        if(y_pred_2[i] == -1):
            result_matrix2[3,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix2[3,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix2[3,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix2[3,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix2[3,3] += 1
    if(y_test[i] == 60): # If its true label is y
        if(y_pred_2[i] == -1):
            result_matrix2[4,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix2[4,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix2[4,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix2[4,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix2[4,3] += 1
    if(y_test[i] == 61): # If its true label is z
        if(y_pred_2[i] == -1):
            result_matrix2[5,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix2[5,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix2[5,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix2[5,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix2[5,3] += 1
    if(y_test[i] == 0): # If its true label is 1
        if(y_pred_2[i] == -1):
            result_matrix2[6,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix2[6,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix2[6,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix2[6,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix2[6,3] += 1
    if(y_test[i] == 1): # If its true label is 3
        if(y_pred_2[i] == -1):
            result_matrix2[7,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix2[7,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix2[7,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix2[7,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix2[7,3] += 1
    if(y_test[i] == 2): # If its true label is 5
        if(y_pred_2[i] == -1):
            result_matrix2[8,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix2[8,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix2[8,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix2[8,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix2[8,3] += 1
    if(y_test[i] == 3): # If its true label is 7
        if(y_pred_2[i] == -1):
            result_matrix2[9,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix2[9,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix2[9,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix2[9,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix2[9,3] += 1

print(result_matrix2)
print(np.sum(result_matrix2))

In [ ]:
# Create plot for MDWSVM-ad:
result_matrix3 = np.zeros((10,5)) # 10 input class, 5 predict class
for i in range(8000):
    if(y_test[i] == 56): # If its true label is u
        if(y_pred_2[i] == 4):
            result_matrix3[0,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix3[0,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix3[0,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix3[0,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix3[0,3] += 1
    if(y_test[i] == 57): # If its true label is v
        if(y_pred_2[i] == 4):
            result_matrix3[1,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix3[1,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix3[1,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix3[1,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix3[1,3] += 1
    if(y_test[i] == 58): # If its true label is w
        if(y_pred_2[i] == 4):
            result_matrix3[2,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix3[2,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix3[2,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix3[2,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix3[2,3] += 1
    if(y_test[i] == 59): # If its true label is x
        if(y_pred_2[i] == 4):
            result_matrix3[3,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix3[3,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix3[3,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix3[3,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix3[3,3] += 1
    if(y_test[i] == 60): # If its true label is y
        if(y_pred_2[i] == 4):
            result_matrix3[4,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix3[4,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix3[4,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix3[4,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix3[4,3] += 1
    if(y_test[i] == 61): # If its true label is z
        if(y_pred_2[i] == 4):
            result_matrix3[5,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix3[5,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix3[5,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix3[5,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix3[5,3] += 1
    if(y_test[i] == 0): # If its true label is 1
        if(y_pred_2[i] == 4):
            result_matrix3[6,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix3[6,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix3[6,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix3[6,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix3[6,3] += 1
    if(y_test[i] == 1): # If its true label is 3
        if(y_pred_2[i] == 4):
            result_matrix3[7,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix3[7,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix3[7,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix3[7,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix3[7,3] += 1
    if(y_test[i] == 2): # If its true label is 5
        if(y_pred_2[i] == 4):
            result_matrix3[8,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix3[8,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix3[8,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix3[8,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix3[8,3] += 1
    if(y_test[i] == 3): # If its true label is 7
        if(y_pred_2[i] == 4):
            result_matrix3[9,4] += 1
        if(y_pred_2[i] == 0):
            result_matrix3[9,0] += 1
        if(y_pred_2[i] == 1):
            result_matrix3[9,1] += 1
        if(y_pred_2[i] == 2):
            result_matrix3[9,2] += 1
        if(y_pred_2[i] == 3):
            result_matrix3[9,3] += 1

print(result_matrix3)
print(np.sum(result_matrix3))

# Return a single plot comparing the distribution of running time of hybrid and MDWSVM_AD (can use box plot)

# Don't specifically talk about how we choose hyperparameters in our report, because we don't know anomaly data in practice and don't have the validation set